In [1]:
import pickle
import pandas as pd
import numpy as np
from azureml.core import Workspace
from azureml.core.model import Model

# Ruta al archivo del modelo
model_path = 'modelo\model.pkl'

# Cargar el modelo desde el archivo .pkl
def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
    return model

# Cargar el modelo
model = load_model(model_path)

# Ejemplo de datos de entrada
# Cambia esto según las características esperadas por tu modelo

age = int(input("Edad: "))
dependents= int(input("Dependientes: "))
occupation = input("Ocupación: ")
city_tier = input("Ciudad: ")
rent=   (input("Renta (₹): "))
loan=   (input("Préstamo (₹): "))
income = (input("Ingreso mensual (₹): "))
disposible_income = (input("Ingreso disponible (₹): "))
gasto_fijo = (input("Gastos fijos (₹): "))
gasto_variable =  (input("Gastos variables (₹): "))

input_data = np.array([[age, dependents, occupation, city_tier, rent, loan, income, disposible_income, gasto_fijo, gasto_variable]])

# Hacer la predicción
predictions = model.predict(input_data)

# Hacer predicciones
#predictions = model.predict(age,dependents,occupation,city_tier,rent,loan,income,disposible_income,gasto_fijo,gasto_variable)

# Mostrar resultados
print("Predicciones:", predictions)


Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (numpy 1.26.4 (c:\users\usuario\documents\github\proyecto_final\azure\lib\site-packages), Requirement.parse('numpy!=1.19.4,<1.24; sys_platform == "win32"'), {'azureml-dataset-runtime'}).


AttributeError: 'ellipsis' object has no attribute 'predict'

In [1]:
import joblib

def load_model(path):
    try:
        model = joblib.load(path)
        print("Modelo cargado correctamente.")
        return model
    except Exception as e:
        print("Error al cargar el modelo:", e)
        return None

# Ruta del modelo descargado de Azure
model_path = "ruta/del/modelo.pkl"  # Asegúrate de poner la ruta correcta

# Cargar el modelo
model = load_model(model_path)

# Verificar si el modelo se cargó bien
if model is None:
    print("El modelo no se pudo cargar correctamente.")
else:
    print("Modelo cargado:", type(model))


Error al cargar el modelo: [Errno 2] No such file or directory: 'ruta/del/modelo.pkl'
El modelo no se pudo cargar correctamente.


pip install azureml-core azureml-defaults scikit-learn

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

In [2]:
df=pd.read_csv('datos_modelo.csv')
df

,Income,Age,Dependents,Occupation,City_Tier,Rent,Loan_Repayment,Desired_Savings,Disposable_Income,Gasto_Fijo,Gastos_variables
0,44637.25,49,0,Self_Employed,Tier_1,13391.17,0.00,6200.54,11265.63,24893.40,8478.21
1,26858.60,34,2,Retired,Tier_2,5371.72,0.00,1923.18,9676.82,12154.42,5027.35
2,50367.61,35,1,Student,Tier_3,7555.14,4612.10,7050.36,13891.45,22451.59,9412.47
3,101455.60,21,0,Self_Employed,Tier_3,15218.34,6809.44,16694.97,31617.95,41904.04,21124.17
4,24875.28,52,4,Professional,Tier_2,4975.06,3112.61,1874.10,6265.70,11366.68,4130.32
...,...,...,...,...,...,...,...,...,...,...,...
19995,40913.47,51,4,Self_Employed,Tier_1,12274.04,7703.85,1163.32,1163.32,24323.79,7722.50
19996,90295.77,21,1,Student,Tier_2,18059.15,0.00,10613.59,30725.18,43865.50,15705.09
19997,40604.57,30,1,Professional,Tier_2,8120.91,8089.61,2267.91,2267.91,22091.78,8155.27
19998,118157.82,27,2,Professional,Tier_1,35447.35,12345.91,10603.68,10603.68,71061.02,24147.20


In [3]:
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType

data_sample = PandasParameterType(pd.DataFrame({"Income": pd.Series([44637.25], dtype="float64"), "Age": pd.Series([49], dtype="int8"), "Dependents": pd.Series([0], dtype="int8"), "Occupation": pd.Series(["Self_Employed"], dtype="object"), "City_Tier": pd.Series(["Tier_1"], dtype="object"), "Rent": pd.Series([13391.17], dtype="float64"), "Loan_Repayment": pd.Series([0], dtype="float64"), "Disposable_Income": pd.Series([11265.63], dtype="float64"), "Gasto_Fijo": pd.Series([24893.4], dtype="float64"), "Gastos_variables": pd.Series([8478.21], dtype="float64")}))
input_sample = StandardPythonParameterType({'data': data_sample})

result_sample = NumpyParameterType(np.array([0.0]))
output_sample = StandardPythonParameterType({'Results':result_sample})
sample_global_parameters = StandardPythonParameterType(1.0)

try:
    log_server.enable_telemetry(INSTRUMENTATION_KEY)
    log_server.set_verbosity('INFO')
    logger = logging.getLogger('azureml.automl.core.scoring_script_v2')
except:
    pass


def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = "modelo\model.pkl"
    path = os.path.normpath(model_path)
    try:
        logger.info("Loading model from path.")
        model = joblib.load(model_path)
        logger.info("Loading successful.")
    except Exception as e:
        logging_utilities.log_traceback(e, logger)
        raise

@input_schema('Inputs', input_sample)
@input_schema('GlobalParameters', sample_global_parameters, convert_to_provided_type=False)
@output_schema(output_sample)
def run(Inputs, GlobalParameters=1.0):
    data = Inputs['data']
    result = model.predict(data)
    return {'Results':result.tolist()}

In [4]:
# Definir los datos de entrada en el mismo formato que `data_sample`
input_data = {
    'data': pd.DataFrame({
        "Income": [44637.25], 
        "Age": [49], 
        "Dependents": [0], 
        "Occupation": ["Self_Employed"], 
        "City_Tier": ["Tier_1"], 
        "Rent": [13391.17], 
        "Loan_Repayment": [0], 
        "Disposable_Income": [11265.63], 
        "Gasto_Fijo": [24893.40], 
        "Gastos_variables": [8478.21]
    })
}

init()
# Llamar a la función run() para obtener predicciones
prediccion = run(input_data)
print(round(prediccion['Results'][0],2))


AttributeError: 'ellipsis' object has no attribute 'predict'